In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import requests
import re

import requests
from bs4 import BeautifulSoup, Comment
import time

import pickle

### Get Pac-12 games and results by season:

Source: Sports-Reference.com

In [4]:
# Get list of urls by season:

u = 'https://www.sports-reference.com/cbb/conferences/pac-12/'
r = requests.get(u).content
soup = BeautifulSoup(r,'lxml')

In [10]:
cols = ['Season','Schls','W','L','W-L%','SRS','SOS','AP','NCAA','Regular Season Champ','Tournament Champ','POY']
master_df = pd.DataFrame(columns = cols + ['url'])

tbls = soup.findAll('table', {'class': 'sortable stats_table'})
for t in tbls:
    for tr in t.find_all('tr')[1:]:
        tds = tr.find_all('td')
        
        new_df = pd.DataFrame([[elem.text for elem in tds]], columns=cols)
        new_df['url'] = tds[0].find('a').get('href')
        master_df = master_df.append(new_df)

In [28]:
master_df.head()

,Season,Schls,W,L,W-L%,SRS,SOS,AP,NCAA,Regular Season Champ,Tournament Champ,POY,url
0,2018-19 Pac-12,12,152,118,.563,7.35,4.49,0,,,,,/cbb/conferences/pac-12/2019.html
0,2017-18 Pac-12,12,231,172,.573,8.62,6.15,1,3,Arizona,Arizona,DeAndre Ayton,/cbb/conferences/pac-12/2018.html
0,2016-17 Pac-12,12,238,168,.586,9.33,6.73,3,4,"Arizona, Oregon",Arizona,Dillon Brooks,/cbb/conferences/pac-12/2017.html
0,2015-16 Pac-12,12,241,160,.601,11.85,8.52,4,7,Oregon,Oregon,Jakob Poeltl,/cbb/conferences/pac-12/2016.html
0,2014-15 Pac-12,12,242,166,.593,9.74,6.59,2,4,Arizona,Arizona,Joseph Young,/cbb/conferences/pac-12/2015.html


In [13]:
url_base = 'https://www.sports-reference.com/'
url_cap = '-schedule.html'
url_tourney_cap = '.html'

In [14]:
def date_parser(s):
    return s[5:-6].strip()

def date_parser_tourney(s):
    return s[5:].strip()

Get games for each season

In [52]:
%%time

# MASTER ITERATION:

game_cols = ['Date', 'Away', 'Away_Pts', 'Home', 'Home_Pts', 'OT', 'Notes']
games_df = pd.DataFrame(columns=game_cols + ['Conference','Year'])
tourney_cols = ['Date','Away','Away_Pts','At','Home','Home_Pts','OT','Box']
        
for index, row in master_df.iterrows():
    conf_df = pd.DataFrame(columns=game_cols + ['Conference','Year'])
    
    url = str(row['url'])[1:-5]
        
    time.sleep(5)
        
    # **** GET LIST OF CONFERENCE GAMES ****
        
    new_df = pd.DataFrame(columns=game_cols)
        
    u = url_base + url + url_cap
    y = url[-4:] # get year from url
        
    u_tourney = url_base + url + url_tourney_cap # url for conference tournaments
        
        
    ### Get main table of games:
    r = requests.get(u).content
    soup = BeautifulSoup(r,'lxml')
        
    if 'Page Not Found' in soup.find('title').text:
        continue
        
    games_tbl = soup.find('table', {'id': 'schedule'})
        
    # iterate through every game on table:
    for tr in games_tbl.findAll('tr'):
        tds = tr.findAll(['th', 'td']) # first column = th tags
            
        temp_df = pd.DataFrame([[elem.text for elem in tds]], columns=game_cols)
        new_df = new_df.append(temp_df)
            
    # remove duplicate header rows and top header row
    new_df = new_df.drop_duplicates()[1:]
    new_df['Conference'] = row[0]
    new_df['Year'] = y
        
        
    # **** GET LIST OF TOURNAMENT GAMES ****
    r_tourney = requests.get(u_tourney).content
    soup_tourney = BeautifulSoup(r_tourney, 'lxml')
        
    tourney_df = pd.DataFrame(columns=tourney_cols)
    comments = soup_tourney.findAll(text=lambda text:isinstance(text, Comment))

    for c in comments:
        comment_soup = BeautifulSoup(comments[16], 'lxml')
        tourney_tbl = comment_soup.find('table', {'id': 'conference-tournament'})
        if tourney_tbl is not None:
            break
    
    # iterate through every game on table:
    try: 
        trs = tourney_tbl.findAll('tr')
        for tr in trs:
            tds = tr.findAll(['th', 'td']) # first column = th tags
            try:
                tourney_temp = pd.DataFrame([[elem.text for elem in tds]], columns=tourney_cols)
            except:
                tourney_temp = pd.DataFrame([[elem.text for elem in tds]], columns=tourney_cols[:-1])
            tourney_df = tourney_df.append(tourney_temp)
    except AttributeError: 
        pass
            
        
    # **** MERGE AND DROP CONFERENCE TOURNAMENT GAMES ****
        
    new_df['Date_clean'] = new_df['Date'].apply(lambda x: date_parser(str(x)))
        
    tourney_df['Date_clean'] = tourney_df['Date'].apply(lambda x: date_parser_tourney(str(x)))
    tourney_df['Tourney']='Y'
    tourney_days = tourney_df[['Date_clean','Tourney']].drop_duplicates()
        
    new_df = new_df.merge(tourney_days, on='Date_clean', how='left')
    new_df = new_df[new_df['Tourney']!="Y"].drop(['Date_clean','Tourney'], axis=1)
        
    # ADD TO MASTER:
        
    conf_df = conf_df.append(new_df)
        
        
    games_df = games_df.append(conf_df)

games_df.to_csv('p12_conference_games.csv', index=False)

Wall time: 9min 55s
